In [1]:
import yaml
import os

from pyspark import SparkContext
import numpy as np
import pickle
import sys
import csv

import ankus4passDALib.unsupervised.kmeans_pp_cluster as kmeans_pp_cluster
import findspark

if __name__ == "__main__":
    bath_path = os.path.dirname(os.path.abspath('__file__'))
    print bath_path
    yaml_file = open(bath_path + '/../etc/spark_config_local.yaml', 'r')

    spark_yaml = yaml.load(yaml_file)
    install_path = spark_yaml["spark_env"]
    SPARK_LOCAL_IP = spark_yaml["SPARK_LOCAL_IP"]
    PYSPARK_DRIVER_PYTHON = spark_yaml["PYSPARK_DRIVER_PYTHON"]
    PYSPARK_DRIVER_PYTHON_OPTS = spark_yaml["PYSPARK_DRIVER_PYTHON_OPTS"]

    os.environ["SPARK_HOME"] = install_path
    os.environ["SPARK_LOCAL_IP"] = SPARK_LOCAL_IP
    os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_DRIVER_PYTHON
    os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = PYSPARK_DRIVER_PYTHON_OPTS
    os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"] = 'YES'  # for mac fork error
    findspark.init()
    bath_path = os.path.dirname(os.path.abspath('__file__'))
    model_path = '../model/iris.pkl'
    input_path = bath_path + '/../DataSet/iris.csv'
    if len(sys.argv) == 2:
        model_path = sys.argv[1]
        input_path = sys.argv[2]
    else:
        print 'Arguments is not valid'
        print 'Run with default argument'
        print 'Model Path ' , model_path
        print 'Input Path' , input_path

    #model pkl load
    with open(model_path, 'rb') as f:
        centroid = pickle.load(f)

    vector_list = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
    object_name = ['class']

    input_raw_data_list = []
    with open(input_path, "r") as f:
        mycsv = csv.DictReader(f)
        for row in mycsv:
            vector_values = []
            for col in vector_list:
                vector_values.extend([row[col]])
            vector_values_np = np.array(list(map(float, vector_values)))
            for col in object_name:
                input_raw_data_list.append((row[col], vector_values_np))

    sc = SparkContext(appName="pyspark_kmeans_test")
    rdd = sc.parallelize(input_raw_data_list)  # 'str type object name, numpy vector'

    print rdd.take(1)

    cluster = kmeans_pp_cluster
    kmeans = cluster.kmeans_pp()

    group2 = rdd.map(
        lambda (in_vector): (in_vector[0], 'cluster:' + str(kmeans.get_closest_centers(in_vector[1], centroid)))).collect()
    for element in group2:
        print element

    sc.stop()

/Users/prismdata/Documents/ankus_analyzer_p_workspace/Bigdata_ML_Library/ankus4pass_DLib/ankus4passDALib/example
Arguments is not valid
Run with default argument
Model Path  ../model/iris.pkl
Input Path /Users/prismdata/Documents/ankus_analyzer_p_workspace/Bigdata_ML_Library/ankus4pass_DLib/ankus4passDALib/example/../DataSet/iris.csv


/Users/prismdata/Documents/ankus_analyzer_p_workspace/Bigdata_ML_Library/ankus4pass_DLib/ankus4pass_env_py2/lib/python2.7/site-packages/ipykernel_launcher.py:18: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


[('Iris-setosa', array([5.1, 3.5, 1.4, 0.2]))]


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.lang.IllegalArgumentException
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.spark.util.ClosureCleaner$.getClassReader(ClosureCleaner.scala:46)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:449)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:432)
	at scala.collection.TraversableLike$WithFilter$$anonfun$foreach$1.apply(TraversableLike.scala:733)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashTable$class.foreachEntry(HashTable.scala:230)
	at scala.colle